In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

MAX_WIDTH = 1280

def display_image(image, title="Image"):
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.axis('off')
    plt.show()

def preprocess_image(image):

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Stage 2: CLAHE
    clahe = cv2.createCLAHE(clipLimit=1.0, tileGridSize=(16, 16))
    equalized = clahe.apply(gray)

    # Add edge sharpening after CLAHE
    kernel_sharpen = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    sharpened = cv2.filter2D(equalized, -1, kernel_sharpen)

    # Stage 3: Blur (using GaussianBlur)
    blurred = cv2.GaussianBlur(equalized, (5, 5), 0)

    # Stage 4: Adaptive Thresholding (using sharpened image)
    adaptive_thresh = cv2.adaptiveThreshold(equalized, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                            cv2.THRESH_BINARY, 15, 5)

    # Stage 5: Morphological Closing (to close small holes in card edges)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (4, 4))
    morphed = cv2.morphologyEx(adaptive_thresh, cv2.MORPH_CLOSE, kernel)

    # Stage 6: Canny Edge Detection
    edged = cv2.Canny(morphed, 70, 150)

    # Add Sobel edge detection
    sobel_x = cv2.Sobel(morphed, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(morphed, cv2.CV_64F, 0, 1, ksize=3)
    sobel_combined = cv2.magnitude(sobel_x, sobel_y)
    sobel_combined = cv2.convertScaleAbs(sobel_combined)

    # Scale down Sobel edges to reduce their influence
    sobel_combined = cv2.multiply(sobel_combined, 0.5)

    # Combine Canny and scaled Sobel edges
    combined_edges = cv2.bitwise_or(edged, sobel_combined)

    # Additional morphological closing to refine edges
    kernel_final = cv2.getStructuringElement(cv2.MORPH_RECT, (11, 11))
    proc_image = cv2.morphologyEx(combined_edges, cv2.MORPH_CLOSE, kernel_final)

    return proc_image


def find_contours(processed_image, min_contour_area=2000):
    # Find contours
    contours, _ = cv2.findContours(processed_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Filter contours based on size
    contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_contour_area]

    return contours


def clean_mask(mask):
    # Define a kernel for morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7))

    # Apply morphological opening to remove small noise
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

    # Apply morphological closing to fill small gaps
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    return mask

def color_based_masking(image):
    # Convert the image to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define color ranges for the cards (adjust these ranges as needed)
    green_lower = np.array([35, 50, 50])
    green_upper = np.array([85, 255, 255])

    red_lower1 = np.array([0, 50, 50])
    red_upper1 = np.array([10, 255, 255])
    red_lower2 = np.array([170, 50, 50])
    red_upper2 = np.array([180, 255, 255])

    yellow_lower = np.array([20, 50, 50])
    yellow_upper = np.array([30, 255, 255])

    blue_lower = np.array([100, 50, 50])
    blue_upper = np.array([130, 255, 255])

    # Create masks for each color
    green_mask = cv2.inRange(hsv, green_lower, green_upper)
    red_mask1 = cv2.inRange(hsv, red_lower1, red_upper1)
    red_mask2 = cv2.inRange(hsv, red_lower2, red_upper2)
    yellow_mask = cv2.inRange(hsv, yellow_lower, yellow_upper)
    blue_mask = cv2.inRange(hsv, blue_lower, blue_upper)

    # Combine all masks
    combined_mask = cv2.bitwise_or(green_mask, red_mask1)
    combined_mask = cv2.bitwise_or(combined_mask, red_mask2)
    combined_mask = cv2.bitwise_or(combined_mask, yellow_mask)
    combined_mask = cv2.bitwise_or(combined_mask, blue_mask)

    combined_mask = clean_mask(combined_mask)

    # Apply the mask to the original image
    result = cv2.bitwise_and(image, image, mask=combined_mask)

    # Find contours on the mask
    contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


    return contours, result

def extract_and_display_cards(contours, original_image, min_area=2000):
    cards = []
    for i, contour in enumerate(contours):
        # Get the bounding box for the contour
        x, y, w, h = cv2.boundingRect(contour)

        # Filter out small contours based on area
        if w * h < min_area:
            continue

        # Crop the card from the original image
        card = original_image[y:y+h, x:x+w]

        # Ensure the longer side is upright
        if w > h:
            card = cv2.rotate(card, cv2.ROTATE_90_CLOCKWISE)

        # Append the card to the list
        cards.append(card)

        # Display the cropped card
        display_image(card, f"Card {i+1}")

    return cards

def draw_bounding_boxes_and_extract_cards(contours, original_image, min_area=10000):
    cards = []
    image_with_boxes = original_image.copy()

    for i, contour in enumerate(contours):
        # Get the minimum area rectangle
        rect = cv2.minAreaRect(contour)
        box = cv2.boxPoints(rect)
        box = np.int32(box)

        # Filter out small contours based on area
        width, height = rect[1]
        if width * height < min_area:
            continue

        # Draw the rotated bounding box on the image
        cv2.drawContours(image_with_boxes, [box], 0, (0, 255, 0), 2)

        # Warp the card to align it properly
        width, height = int(width), int(height)
        src_pts = np.array(box, dtype="float32")
        dst_pts = np.array([[0, 0], [width-1, 0], [width-1, height-1], [0, height-1]], dtype="float32")
        M = cv2.getPerspectiveTransform(src_pts, dst_pts)
        warped_card = cv2.warpPerspective(original_image, M, (width, height))

        # Ensure the longer side is upright
        if height > width:
            warped_card = cv2.rotate(warped_card, cv2.ROTATE_90_CLOCKWISE)

        # Append the card to the list
        cards.append(warped_card)

        # Display the warped card
        display_image(warped_card, f"Card {i+1}")

    # Display the image with bounding boxes
    display_image(image_with_boxes, "Bounding Boxes on Original Image")

    return cards

if __name__ == "__main__":
    image = cv2.imread("pictures/uno8.jpeg")

    # Resize image if it's too wide
    aspect_ratio = image.shape[0] / image.shape[1]
    new_width = MAX_WIDTH
    new_height = int(aspect_ratio * new_width)
    image = cv2.resize(image, (new_width, new_height))

    display_image(image, "Resized Image")

    contours, masked_image = color_based_masking(image)
    display_image(masked_image, "Masked Image")

    # apply contours to the original image
    contoured_image = image.copy()
    cv2.drawContours(contoured_image, contours, -1, (0, 255, 0), 3)
    display_image(contoured_image, "Contours on Original Image")


    #processed_image = preprocess_image(image)
    #display_image(processed_image, "Processed Image")

    draw_bounding_boxes_and_extract_cards(contours, image)
